# **Skin Anomaly Detection**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pip install pandas numpy opencv-python tensorflow scikit-learn

In [6]:
import os
import zipfile
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, fbeta_score

In [7]:
# Extract zip files
zip_paths = ['/content/drive/MyDrive/Skeenai/HAM 100000/HAM10000_images_part_1.zip', '/content/drive/MyDrive/Skeenai/HAM 100000/HAM10000_images_part_2.zip']
extract_path = '/content/drive/MyDrive/Skeenai/HAM 100000/Extracted'
for zip_path in zip_paths:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Load the metadata
metadata_path = '/content/drive/MyDrive/Skeenai/HAM 100000/HAM10000_metadata'
metadata = pd.read_csv(metadata_path)

In [8]:
# Load images
def load_images(metadata, image_dir):
    images = []
    labels = []
    for index, row in metadata.iterrows():
        image_path = os.path.join(image_dir, row['image_id'] + '.jpg')
        if os.path.exists(image_path):
            image = cv2.imread(image_path)
            image = cv2.resize(image, (64, 64))  # Resize to 64x64 pixels
            images.append(image)
            labels.append(row['dx'])
    return np.array(images), np.array(labels)

images, labels = load_images(metadata, extract_path)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [9]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20
251/251 [==============================] - 10s 13ms/step - loss: 2.5566 - accuracy: 0.6444 - val_loss: 0.9307 - val_accuracy: 0.6695
Epoch 2/20
251/251 [==============================] - 2s 7ms/step - loss: 0.9746 - accuracy: 0.6736 - val_loss: 0.9069 - val_accuracy: 0.6625
Epoch 3/20
251/251 [==============================] - 2s 6ms/step - loss: 0.9100 - accuracy: 0.6735 - val_loss: 0.8870 - val_accuracy: 0.6645
Epoch 4/20
251/251 [==============================] - 2s 6ms/step - loss: 0.8916 - accuracy: 0.6852 - val_loss: 0.8964 - val_accuracy: 0.6790
Epoch 5/20
251/251 [==============================] - 2s 6ms/step - loss: 0.8583 - accuracy: 0.6882 - val_loss: 0.8579 - val_accuracy: 0.6885
Epoch 6/20
251/251 [==============================] - 2s 7ms/step - loss: 0.8354 - accuracy: 0.6938 - val_loss: 0.8570 - val_accuracy: 0.6870
Epoch 7/20
251/251 [==============================] - 2s 6ms/step - loss: 0.8308 - accuracy: 0.6982 - val_loss: 0.8780 - val_accuracy: 0.6940
Epoc

In [10]:
# Predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)

# Precision
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')

# Recall
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')

# F1 Score
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

# F2 Score
f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')

# F0.5 Score
f0_5 = fbeta_score(y_true_classes, y_pred_classes, beta=0.5, average='weighted')

# Classification report
report = classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print(f"F2 Score: {f2 * 100:.2f}%")
print(f"F0.5 Score: {f0_5 * 100:.2f}%")
print("\nClassification Report:\n", report)

63/63 [==============================] - 0s 2ms/step
Accuracy: 70.29%
Precision: 63.50%
Recall: 70.29%
F1 Score: 64.74%
F2 Score: 67.77%
F0.5 Score: 62.97%

Classification Report:
               precision    recall  f1-score   support

       akiec       0.00      0.00      0.00        69
         bcc       0.38      0.22      0.27        93
         bkl       0.38      0.36      0.37       228
          df       0.00      0.00      0.00        28
         mel       0.51      0.12      0.19       226
          nv       0.76      0.95      0.85      1338
        vasc       0.69      0.52      0.59        21

    accuracy                           0.70      2003
   macro avg       0.39      0.31      0.33      2003
weighted avg       0.64      0.70      0.65      2003



In [11]:
# Define the save path
save_path = '/content/drive/MyDrive/Skeenai/HAM 100000/skin_disease_detection_model.h5'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the model
model.save(save_path)

print(f"Model saved to {save_path}")

Model saved to /content/drive/MyDrive/Skeenai/HAM 100000/skin_disease_detection_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# **Skin Acne Detection**

In [16]:
import os
import zipfile
import glob
import re
import json
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, classification_report
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from google.colab import drive

In [17]:
# Define paths
zip_path = '/content/drive/MyDrive/Skeenai/ACNE 04/ACNE04 (Manuel).zip'
extract_path = '/content/drive/MyDrive/Skeenai/ACNE 04/extracted'
small_1024_path = os.path.join(extract_path, 'acne_1024', 'small_1024')
renamed_image_dir = os.path.join(extract_path, 'acne_1024', 'small_1024_renamed')
save_path = os.path.join(extract_path, 'beauty_issues_detection_model.h5')

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Rename files based on a mapping
def rename_files(directory_path, rename_map):
    for filename in os.listdir(directory_path):
        if filename.endswith('.jpg'):
            for original, new in rename_map.items():
                if original in filename:
                    new_filename = filename.replace(original, new)
                    old_file_path = os.path.join(directory_path, filename)
                    new_file_path = os.path.join(directory_path, new_filename)
                    os.rename(old_file_path, new_file_path)
                    print(f'Renamed "{filename}" to "{new_filename}"')
                    break  # Stop checking after the first match
    print("Renaming complete.")

rename_map = {
    'acne0': 'acnezero',
    'acne1': 'acnesmall',
    'acne2': 'acnemedium',
    'acne3': 'acnestrong'
}

rename_files(small_1024_path, rename_map)

Renaming complete.


In [18]:
# Generate metadata.jsonl
def generate_metadata(image_dir):
    base_prompt = "photo of a person with acne"
    jpg_files = glob.glob(f"{image_dir}/**/*.jpg", recursive=True)
    metadata_list = []

    for jpg_file in jpg_files:
        filename = jpg_file.split('/')[-1]
        match = re.search(r'(acnezero|acnesmall|acnemedium|acnestrong)', filename)
        if match:
            level = match.group(1)
            prompt = f"{base_prompt} {level}"
            entry = {"file_name": jpg_file, "prompt": prompt, "level": level}
            metadata_list.append(entry)

    metadata_file_path = os.path.join(image_dir, 'metadata.jsonl')
    with open(metadata_file_path, 'w') as file:
        for entry in metadata_list:
            file.write(json.dumps(entry) + '\n')

    return metadata_file_path

metadata_file_path = generate_metadata(renamed_image_dir)

# Load metadata
def load_metadata(metadata_file_path):
    images = []
    labels = []

    with open(metadata_file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            image_path = entry["file_name"]
            level = entry["level"]
            if os.path.exists(image_path):
                image = cv2.imread(image_path)
                image = cv2.resize(image, (64, 64))  # Resize to 64x64 pixels
                images.append(image)
                labels.append(level)

    return np.array(images), np.array(labels)

images, labels = load_metadata(metadata_file_path)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [19]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
5/5 [==============================] - 2s 132ms/step - loss: 38.2980 - accuracy: 0.2875 - val_loss: 4.2192 - val_accuracy: 0.3250
Epoch 2/20
5/5 [==============================] - 0s 13ms/step - loss: 8.4995 - accuracy: 0.2688 - val_loss: 2.4693 - val_accuracy: 0.2250
Epoch 3/20
5/5 [==============================] - 0s 17ms/step - loss: 2.0436 - accuracy: 0.3000 - val_loss: 1.3402 - val_accuracy: 0.4000
Epoch 4/20
5/5 [==============================] - 0s 13ms/step - loss: 1.3525 - accuracy: 0.3375 - val_loss: 1.4099 - val_accuracy: 0.3000
Epoch 5/20
5/5 [==============================] - 0s 12ms/step - loss: 1.2613 - accuracy: 0.4375 - val_loss: 1.4068 - val_accuracy: 0.3750
Epoch 6/20
5/5 [==============================] - 0s 16ms/step - loss: 1.1685 - accuracy: 0.4500 - val_loss: 1.4080 - val_accuracy: 0.3750
Epoch 7/20
5/5 [==============================] - 0s 16ms/step - loss: 1.0916 - accuracy: 0.5125 - val_loss: 1.4021 - val_accuracy: 0.3250
Epoch 8/20
5/5 [=========

In [21]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Metrics calculation
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
f0_5 = fbeta_score(y_true_classes, y_pred_classes, beta=0.5, average='weighted')
classification_report_str = classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print(f"F2 Score: {f2 * 100:.2f}%")
print(f"F0.5 Score: {f0_5 * 100:.2f}%")
print("\nClassification Report:\n", classification_report_str)

2/2 [==============================] - 0s 7ms/step
Accuracy: 40.00%
Precision: 38.91%
Recall: 40.00%
F1 Score: 37.66%
F2 Score: 38.48%
F0.5 Score: 38.07%

Classification Report:
               precision    recall  f1-score   support

  acnemedium       0.25      0.17      0.20         6
   acnesmall       0.50      0.40      0.44        15
  acnestrong       0.41      0.78      0.54         9
    acnezero       0.29      0.20      0.24        10

    accuracy                           0.40        40
   macro avg       0.36      0.39      0.35        40
weighted avg       0.39      0.40      0.38        40



In [20]:
# Define the save path
save_path = os.path.join(extract_path, 'beauty_issues_detection_model.h5')

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the model
model.save(save_path)

print(f"Model saved to {save_path}")

Model saved to /content/drive/MyDrive/Skeenai/ACNE 04/extracted/beauty_issues_detection_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
